In [1]:
from __future__ import division

import os
import random as rnd

import numpy as np
import scipy as sp
import skimage as ski
from matplotlib import pylab as plt
from pylab import *
from skimage import exposure
from skimage import img_as_float
from skimage import io, filters
from skimage import measure
from skimage import restoration
from skimage.color import rgb2gray
from skimage.color import rgb2hed
from skimage.morphology import remove_small_objects, convex_hull_image
from skimage.transform import hough_line, hough_line_peaks, probabilistic_hough_line, hough_circle, hough_circle_peaks
from skimage.feature import canny

import cv2

In [2]:
def prepare_img(img):
    ihc_hed = rgb2hed(img) # zmiana na format hed w celu poprawy kontrastu
    h = exposure.rescale_intensity(ihc_hed[:, :, 0], out_range=(0, 1))
    d = exposure.rescale_intensity(ihc_hed[:, :, 2], out_range=(0, 1))
    zdh = np.dstack((np.zeros_like(h), d, h)) # połaczenie wysokiego i niskiego kontrastu
    tmp = rgb2gray(zdh)
    p0_5, p99_5 = np.percentile(tmp, (0.5, 99.5))
    tmp = exposure.rescale_intensity(tmp, (p0_5, p99_5)) # poprawa kontrastu według percentyli
    tmp = filters.prewitt(tmp) 
    tmp = restoration.denoise_nl_means(tmp, h=0.95, fast_mode=True, patch_size=3, patch_distance=7, multichannel=False) # odszumieanie z użyciem średniej
    return tmp


def thresh_img(prepared_img):
    try:
        binary = (prepared_img > filters.threshold_minimum(prepared_img)) # progowanie oparte na metodzie minimum
    except:
        binary = (prepared_img > filters.threshold_mean(prepared_img)) # awaryjne progowanie dla przypadku jednoskośnego lub sbyt płaskiego histogramu
    return binary


def outline_img(img):
    tmp = filters.prewitt(img)
#     tmp_outline = filters.sobel(tmp)
#     tmp_outline = canny(tmp)
#     tmp_outline = sp.ndimage.binary_fill_holes(tmp_outline)
#     tmp_outline = remove_small_objects(tmp_outline, 20 * 20)
    return tmp


def get_solidities_and_boundings(outlined_img):
    outlines, count = sp.ndimage.label(outlined_img)
    outlines = measure.regionprops(outlines) 
    solidities = []
    boundings = []
    for outline in outlines:
        solidity  = outline.solidity
        solidities.append(solidity)
        x1, y1, x2, y2 = outline.bbox
        boundings.append([(x1, y1), (x2, y2)])
    return solidities, boundings
        

def find_game_areas(solidities_arr, boundings_arr):
    solidities = np.array(solidities_arr)
    boundings = np.array(boundings_arr)
    game_areas = boundings[solidities < 0.35] # wartość wyznaczona empirycznie 24
    return game_areas


def resize_bounding_box(shape, game_area, resize_multiplier=1):
    delta1 = int((game_area[1, 0] - game_area[0, 0])*(game_area[1, 0] - game_area[0, 0])/shape[0]) * int(resize_multiplier)
    delta2 = int((game_area[1, 1] - game_area[0, 1])*(game_area[1, 1] - game_area[0, 1])/shape[1]) * int(resize_multiplier)
    x1_d = game_area[0, 0] - delta1
    x1 = x1_d if x1_d > 0 else 0
    x2_d = game_area[1, 0] + delta1
    x2 = x2_d if x2_d < shape[0] else shape[0]-1
    y1_d = game_area[0, 1] - delta2
    y1 = y1_d if y1_d > 0 else 0
    y2_d = game_area[1, 1] + delta2
    y2 = y2_d if y2_d < shape[1] else shape[1]-1
    return x1, y1, x2, y2


def find_lines(prep_img):
    tested_angles = np.linspace(-np.pi / 2, np.pi / 2, 360)
    h, theta, d = hough_line(prep_img, theta=tested_angles)
    
    _, angles, dists = hough_line_peaks(h, theta, d, threshold=0.5*max([max(h_n) for h_n in h]), min_distance=100)
    return angles, dists

def line_polar_to_cartesian(angles, dists):
    x = np.array([0, 1])
    lines_factors = []
    for angle, dist in zip(angles, dists):
        y0, y1 = (dist - x*np.cos(angle))/np.sin(angle)
        a = (y0 - y1)/-1
        b = y0
        lines_factors.append([a, b])
    lines_factors = np.array(lines_factors)
    return lines_factors


def equal(x0, x1, precision=0):
    div = np.abs(x0 - x1)
    return div <= precision


# def count_dist(a, b, x, y):
#     return np.absolute(a*x+y+b)/np.sqrt(a**2+1)


def find_parallel_lines(angles, dists, prec):
    parallel_lines = []
    ln = angles.shape[0]
    for i in range(ln-1):
        ang_i = angles[i]
        for j in range(i+1, ln):
            ang_j = angles[j]
            if (equal(ang_i, ang_j, prec)):
                parallel_lines.append([[ang_i, dists[i]], [ang_j, dists[j]]])
    parallel_lines = np.array(parallel_lines)
    return parallel_lines


def find_game_lines(parallel_lines, prec=0):
    game_areas = []
    ln = parallel_lines.shape[0]
    for i in range(ln-1):
        ang_1, dist_1 = parallel_lines[i, 0] 
        ang_2, dist_2 = parallel_lines[i, 1]
        for j in range(i+1, ln):
            ang_3, dist_3 = parallel_lines[j, 0]
            ang_4, dist_4 = parallel_lines[j, 1]
            if(equal(ang_1, np.pi-ang_3, prec) and equal(ang_2, np.pi-ang_4, prec) 
               and equal(ang_1, np.pi-ang_4, prec) and equal(ang_2, np.pi-ang_3, prec)):
                game_areas.append([[ang_1, dist_1], [ang_2, dist_2], [ang_3, dist_3], [ang_4, dist_4]])
    game_areas = np.array(game_areas)
    return game_areas


def get_lines_crossing(l1, l2):
    a = np.array([[l1[0], -1], [l2[0], -1]])
    b = np.array([-l1[1], -l2[1]])
    point  = np.linalg.solve(a, b)
    point_int = np.array([int(point[0]), int(point[1])])
    return point_int


def get_line_eq(p1, p2):
    a = (p1[1] - p2[1])/(p1[0] - p2[0])
    b = p1[1] - a*p1[0]
    return (a, b)


def line_func(x, a, b):
    return a*x+b


def get_mark(img):
    tmp = sp.ndimage.binary_fill_holes(img.astype(int))
    label, count = sp.ndimage.label(tmp)
    region_prop = measure.regionprops(label)
    if(len(region_prop)>0):
        solidity = region_prop[0].solidity
        if(solidity > 0.97):
            return 'O'
        else:
            return 'X'
    else:
        return 'P'


def count_dist(p0, p1):
    dist = np.sqrt((p1[0] - p0[0])**2 + (p1[1] - p0[1])**2)
    return dist
    
    
def get_closest_ind(p0, points):
    dist = np.inf
    closest = 0
    for i in range(len(points)):
        tmp = count_dist(p0, points[i])
        if (tmp < dist):
            dist = tmp
            closest = i
    return closest
   

def find_minmax_vals(points):
    x = []
    y = []
    for point in points:
        x.append(point[0])
        y.append(point[1])
        
    x_min = min(x)
    x_max = max(x)
    y_min = min(y)
    y_max = max(y)
    return [x_min, y_min, x_max, y_max]

In [ ]:
dir = 'zdjęcia'
dir_listed = os.listdir(dir)    # lista plików w folderze

# dir_listed = ['IMG_6617.jpg']

for img_name in dir_listed:
    img = io.imread(os.path.join(dir, img_name))
    fig, ax = subplots(figsize=(50, 100))
    ax.imshow(img)
    ax.set_axis_off()
    plt.show()
    prep_img = prepare_img(img)
    threshed_img = thresh_img(prep_img)
    labels, count = sp.ndimage.label(threshed_img)
    region_props = measure.regionprops(labels)
    centroids = []
    solidities = []
    boundings = []
    for region_prop in region_props:
        centroids.append(region_prop.centroid)
        solidity  = region_prop.solidity
        solidities.append(solidity)
        x1, y1, x2, y2 = region_prop.bbox
        boundings.append([(x1, y1), (x2, y2)])
    games_areas = find_game_areas(solidities, boundings)
    outlined_img = outline_img(threshed_img)
    threshed_img_r = remove_small_objects(threshed_img, 30 * 30)
    sh = outlined_img.shape
    result_img = img
    for game_area in games_areas:
        x1, y1, x2, y2 = resize_bounding_box(sh, game_area)
        temp_img = outlined_img[x1:x2, y1:y2]
        part_img = img[x1:x2, y1:y2]
        th_part = threshed_img_r[x1:x2, y1:y2]
        angles, dists = find_lines(temp_img)
        
#         tmp_lines = line_polar_to_cartesian(angles, dists)
#         fig, ax = subplots(figsize=(100, 50))

#         ax.imshow(temp_img, cmap=plt.cm.gray)
#         x = np.array([0, temp_img.shape[1]])
#         for a, b in tmp_lines:
#             ax.plot(x, line_func(x, a, b), "r")
#         ax.axes.set_xlim(x)
#         ax.set_ylim([temp_img.shape[0], 0])
#         ax.set_axis_off()

#         plt.show()
        
        
        parallel_lines = find_parallel_lines(angles, dists, np.pi/5)
        
#         ax.imshow(temp_img, cmap=plt.cm.gray)
#         x = np.array([0, temp_img.shape[1]])
#         for lines in _lines:
#             for a, b in lines:
#                 ax.plot(x, line_func(x, a, b), "r")
#         ax.axes.set_xlim(x)
#         ax.set_ylim([temp_img.shape[0], 0])
#         ax.set_axis_off()
#         plt.show()
        
        game_lines = find_game_lines(parallel_lines, np.pi)
        
        for i in range(len(game_lines)):
            for j in range(4):
                ang, dist = game_lines[i, j]
                a, b  = line_polar_to_cartesian([ang], [dist])[0]
                game_lines[i, j] = [a, b]
        
        sh_tmp = temp_img.shape
        fields = [] # obliecznie punktów centralnych pól planszy
        try:
            for game in game_lines:
                central_field_corners = []
                for i in range(2):
                    for j in range(2, 4):
                        crossing = get_lines_crossing(game[i], game[j])
                        central_field_corners.append(crossing)

                l1 = get_line_eq(central_field_corners[0], central_field_corners[3])
                l2 = get_line_eq(central_field_corners[2], central_field_corners[1])
                central_field = get_lines_crossing(l1, l2)

                p1 = (central_field_corners[1] + central_field_corners[0])//2 # znajdujemy punkt na środku krawędzi pola centralnego
                v = p1 - central_field # obliczamy wektor między środkiem centralnego pola, a środkiem jego krawędzi
                f2 = p1 + v # "przesuwamy" pole na przybliżony środek pola skrajnego
                # powyższe kroki powtarzamy dla pozostałych trzech pól mających wspólną krawędź z polem centralnym

                p2 = (central_field_corners[3] + central_field_corners[2])//2
                v = p2 - central_field
                f6 = p2 + v

                p3 = (central_field_corners[0] + central_field_corners[2])//2
                v = p3 - central_field
                f8 = p3 + v

                p4 = (central_field_corners[3] + central_field_corners[1])//2
                v = p4 - central_field
                f4 = p4 + v

                # obliczanie przybiżonych środków pól na skos od centralnego pola
                v = central_field_corners[0] - central_field # obliczamy wektor mięszy środkiem pola centralnego a rogiem tego pola
                f1 = central_field_corners[0] + v # "przesuwamy" pole o ten wektor uzyskując przybliżony środek pola

                v = central_field_corners[1] - central_field
                f3 = central_field_corners[1] + v

                v = central_field_corners[3] - central_field
                f5 = central_field_corners[3] + v

                v = central_field_corners[2] - central_field
                f7 = central_field_corners[2] + v

                fields.append(f1)
                fields.append(f2)
                fields.append(f3)
                fields.append(f4)
                fields.append(f5)
                fields.append(f6)
                fields.append(f7)
                fields.append(f8)
                fields.append(central_field)
        except:
            continue
            
#         prep_part_img = prepare_img(part_img)
#         outlined_part_img = thresh_img(prep_part_img)
#         part_img_labels, part_img_count = sp.ndimage.label(outlined_part_img)
        part_img_labels, part_img_count = sp.ndimage.label(th_part)
        part_img_region_props = measure.regionprops(part_img_labels)
        part_img_centroids = []
        part_img_solidities = []
        part_img_bbox_areas = []
        part_img_boundings = []
        for part_img_region_prop in part_img_region_props:
            part_x1, part_y1, part_x2, part_y2 = part_img_region_prop.bbox
            part_img_boundings.append([(part_x1, part_y1), (part_x2, part_y2)])
            part_img_bbox_area = np.absolute(part_x2 - part_x1)*np.absolute(part_y2 - part_y1)
            part_img_bbox_areas.append(part_img_bbox_area)
            part_img_centroids.append(part_img_region_prop.centroid)
            solidity  = part_img_region_prop.solidity
            part_img_solidities.append(solidity)
            
        max_bbox_area = 0
        max_ind = -100
        for i in range(len(part_img_centroids)):
            if(part_img_bbox_areas[i] > max_bbox_area):
                max_bbox_area = part_img_bbox_areas[i]
                max_ind = i
                
        if(max_ind != -100):
            if max_ind < len(part_img_centroids):
                part_img_bbox_areas = part_img_bbox_areas[:max_ind] + part_img_bbox_areas[max_ind+1:]
                part_img_centroids = part_img_centroids[:max_ind] + part_img_centroids[max_ind+1:]
                part_img_solidities = part_img_solidities[:max_ind] + part_img_solidities[max_ind+1:]
                part_img_boundings = part_img_boundings[:max_ind] + part_img_boundings[max_ind+1:]
            else:
                part_img_bbox_areas = part_img_bbox_areas[:max_ind]
                part_img_centroids = part_img_centroids[:max_ind]
                part_img_solidities = part_img_solidities[:max_ind]
                part_img_boundings = part_img_boundings[:max_ind]
        
        marks_on_fields = {}
        for centroid, solidity, bounding in zip(part_img_centroids, part_img_solidities, part_img_boundings):
            ind = get_closest_ind([centroid[1], centroid[0]], fields)
            px1, py1 = bounding[0]
            px2, py2 = bounding[1]
            mark = get_mark(th_part[px1:px2, py1:py2])
            marks_on_fields[str(ind)] = mark
            
            
            
        num = 1
        for i in range(len(fields)):
            if str(i) in marks_on_fields:
                mark = marks_on_fields[str(i)]
            else:
                mark = 'P'
#             cv2.putText(temp_img, str(num), (field[0], field[1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (1.0), 2)
            cv2.putText(part_img, str(num) + ":" + mark, (fields[i][0], fields[i][1]), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 2)
            num += 1
            
#         for centroid, bounding in zip(part_img_centroids, part_img_boundings):
#             px1, py1 = bounding[0]
#             px2, py2 = bounding[1]
#             cv2.putText(part_img, get_mark(th_part[px1:px2, py1:py2]), (int(centroid[1]), int(centroid[0])), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)
        
        result_img[x1:x2, y1:y2] = part_img
        
#         for pairs in game_lines:
#             fig, ax = subplots(figsize=(100, 50))

#             ax.imshow(temp_img, cmap=plt.cm.gray)
#             x = np.array([0, temp_img.shape[1]])
#             for a, b in pairs:
#                 ax.plot(x, line_func(x, a, b), "r")
#             ax.axes.set_xlim(x)
#             ax.set_ylim([temp_img.shape[0], 0])
#             ax.set_axis_off()

#             plt.show()
        
            
    fig, ax = subplots(figsize=(50, 100))

    ax.imshow(result_img)
    ax.set_axis_off()

    plt.show()